<a href="https://colab.research.google.com/github/gagan3012/DetectingBias/blob/master/GPTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q kaggle
!mkdir ~/.kaggle
!echo '{"username":"gagan2000","key":"73e5d9db85677b98fe016b25873727ec"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d snapcrack/all-the-news -p /content

 92% 225M/244M [00:03<00:00, 79.3MB/s]
100% 244M/244M [00:03<00:00, 66.8MB/s]


In [3]:
!unzip /content/all-the-news.zip -d /content/

Archive:  /content/all-the-news.zip
  inflating: /content/articles1.csv  
  inflating: /content/articles2.csv  
  inflating: /content/articles3.csv  


In [4]:
!pip install datasets

     |████████████████████████████████| 184kB 6.4MB/s 
     |████████████████████████████████| 245kB 7.7MB/s 
     |████████████████████████████████| 112kB 9.3MB/s 


In [5]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 64769 (delta 0), reused 0 (delta 0), pack-reused 64766
Receiving objects: 100% (64769/64769), 48.69 MiB | 30.70 MiB/s, done.
Resolving deltas: 100% (45915/45915), done.


In [6]:
!pip install transformers

     |████████████████████████████████| 1.9MB 6.0MB/s 
     |████████████████████████████████| 890kB 20.9MB/s 
     |████████████████████████████████| 3.2MB 34.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a4fc8ae6f5e353442a714a62482a8604da34403eb169f3975bf15ddc4d979666
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [8]:
d1 = pd.read_csv('articles1.csv', encoding = 'utf-8')
d2 = pd.read_csv('articles2.csv', encoding = 'utf-8')
d3 = pd.read_csv('articles3.csv', encoding = 'utf-8')
data = pd.concat([d1, d2, d3])
data.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [9]:
data['content'] = data['content'].fillna("")
data['title'] = data['title'].fillna("")

In [10]:
final_right = data[data['publication'].str.contains("Fox News")]
final_right = final_right[['content']]
final_right.shape

(4354, 1)

In [11]:
final_left = data[data['publication'].str.contains("CNN")]
final_left = final_left[['content']]
final_left.shape

(11488, 1)

In [12]:
final_right['content'].to_csv("rightcontent.txt",header=None, index=None, sep='\t', mode='a')
final_left['content'].to_csv("leftcontent.txt",header=None, index=None, sep='\t', mode='a')

In [13]:
from sklearn.model_selection import train_test_split

with open('/content/rightcontent.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

with open('/content/rightcontent.txt', 'r') as data:
  dataset2 = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset + dataset2, train_size=.9, random_state=2020)
print(len(train))
print(len(eval))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

7837
871


In [17]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./language-modeling")

In [ ]:
!python run_clm.py \
--model_type gpt2 \
--model_name_or_path gpt2 \
--train_file "/content/train_tmp.txt" \
--do_train \
--validation_file "/content/eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/model"

2021-02-27 17:31:23.646930: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/27/2021 17:31:24 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/27/2021 17:31:24 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/model, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Feb27_17-31-24_6c4ab1d28872, logging_first_step=False, logging_steps=500, save_steps=-1, save_total_limit=No